In [1]:
import multiprocessing as mp
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt

In [2]:
e_coli_file = "./archivosParaDotplot/E_coli.fna"
salmonella_file = "./archivosParaDotplot/Salmonella.fna"

In [3]:
def sequence_portions_from_fasta(file_path):
    num_processes = mp.cpu_count()
    sequence_portions = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequence_portions += np.array_split(list(record.seq), num_processes)
    return sequence_portions

In [4]:
def sequence_str_to_uint8(sequence):
    return np.frombuffer(sequence.encode(), dtype=np.uint8)

In [5]:
e_coli_sequence_portions = sequence_portions_from_fasta(e_coli_file)

In [6]:
# Función que llena una porción de la matriz
def fill_matrix_portion(seq1_portion, seq2_file, output_file_name):
    output_file = open(output_file_name, "w")
    for s1 in seq1_portion:
        for record in SeqIO.parse(seq2_file, "fasta"):
            for s2 in record.seq:
                output_file.write(f"{int(s1 == s2)} ")
        output_file.write("\n")
    output_file.close()

In [27]:
# Crea y lanza los procesos
processes = []
for i, e_coli_sequence_portion in enumerate(e_coli_sequence_portions):
    output_file_name = f"outputs/output_{i}.txt"
    process = mp.Process(
        target=fill_matrix_portion,
        args=(e_coli_sequence_portion, salmonella_file, output_file_name),
    )
    processes.append(process)
    process.start()

for process in processes:
    process.join()

KeyboardInterrupt: 

In [10]:
import matplotlib.pyplot as plt
import os
import numpy as np

# Determina la cantidad de archivos de salida generados por el código anterior
num_processes = mp.cpu_count()

# Crea una lista vacía para almacenar las matrices de cada proceso
matrices = []

# Lee cada archivo de salida y añade su contenido a la lista de matrices
for i in range(num_processes):
    output_file_name = f"outputs/output_{i}.txt"
    if os.path.exists(output_file_name):
        with open(output_file_name, "r") as f:
            matrix = []
            for line in f:
                row = [int(cell) for cell in line.strip().split()]
                matrix.append(row)
            matrices.append(matrix)

# Une las matrices en una sola
matrix = np.concatenate(matrices)

# Dibuja el dotplot
plt.imshow(matrix, cmap="Greys", interpolation="nearest")
plt.show()

KeyboardInterrupt: 